In [1]:
# Libraries
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import pygame
import sys
from arrow import draw_arrow
import ast
import random

pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Constants (in inches)
SCALE = 7               # Scale of other constants
W = [60, 44]            # Table dimensions
H = [104, 88]
POCKET_RADIUS = 2.25
BALL_MASS = 210         # Ball specs
BALL_RADIUS = 1.125
MAX_SPEED = 2           # Max speed of ball
FRIC = 0.99             # Friction

In [3]:
MAX_REBOUND = 3         # Maximum number of rebounds
RADIUS_SUM = POCKET_RADIUS + BALL_RADIUS
pos_max = np.array([W[1] - RADIUS_SUM, H[1] - RADIUS_SUM]) * SCALE # Maximum position

# Random initial conditions
def rand_init():
    dir = rnd.randint(0, 360)
    cue_pos = rnd.randint(RADIUS_SUM, pos_max, size=2).astype(float)
    ball_pos = rnd.randint(RADIUS_SUM, pos_max, size=2).astype(float)
    return dir, cue_pos, ball_pos

# Initial conditions with cue ball close to the 8-ball
def close_init(radius=10*SCALE): # Change default parameters later
    dir = rnd.rand() * 2 * np.pi
    cue_pos = rnd.randint(RADIUS_SUM, pos_max, size=2).astype(float)
    ball_pos = np.round(cue_pos + rnd.uniform(2*BALL_RADIUS*SCALE, radius) * np.array([np.cos(dir), np.sin(dir)]))
    # center = rnd.uniform([pos_range[0,0] + radius, pos_range[0,1] - radius], [pos_range[1,0] + radius, pos_range[1,1] - radius], size=2)
    # angles = rnd.rand(2) * 2 * np.pi
    # cue_pos = np.round(center + radius * np.array([np.cos(angles[0]), np.sin(angles[0])]))
    # ball_pos = np.round(center + radius * np.array([np.cos(angles[1]), np.sin(angles[1])]))
    dir = np.round(np.arctan2(ball_pos[1] - cue_pos[1], ball_pos[0] - cue_pos[0]) * 180 / np.pi)
    return dir, cue_pos, ball_pos

In [4]:
# Initialize Pygame
pygame.init()

# Constants
PADDING = SCALE * (W[0]//15)
WIDTH = SCALE * W[0] + PADDING
HEIGHT = SCALE * H[0] + PADDING
X_OFFSET = (WIDTH - SCALE * W[1])//2
Y_OFFSET = (HEIGHT - SCALE * H[1])//2

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BROWN = (139, 69, 19)
RED = (128, 0, 0)
GREEN = (0, 128, 0)
CUE_COLOR = WHITE
BALL_COLOR = BLACK

# Create background
pygame.display.set_caption("Billiards Simulation")
screen = pygame.display.set_mode((WIDTH, HEIGHT))
screen.fill(WHITE)
pockets = np.zeros((6, 2))
for i in range(6):
    x = (i%2) * SCALE * W[1] + X_OFFSET
    y = (i//2) * SCALE * H[1]//2 + Y_OFFSET
    pockets[i] = np.array([x, y])

# Start simulation
file = open("pocketed/speed=2(4).txt", 'a')
running, init, play = True, True, False
while running:
    if init:
        dir, cue_pos, ball_pos = close_init()
        init_cue_pos, init_ball_pos = cue_pos.copy(), ball_pos.copy()
        print(f"Initial conditions: dir = {dir}, cue_pos = {cue_pos}, ball_pos = {ball_pos}")
        cue_vel = np.zeros(2)
        ball_vel = np.zeros(2)
        CUE_COLOR = WHITE
        BALL_COLOR = BLACK
        init = False
        play = False
        
    # keys = pygame.key.get_pressed()
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                init = True
            if event.key == pygame.K_RETURN:
                cue_vel = SCALE * MAX_SPEED * np.array([np.cos(dir * np.pi / 180), np.sin(dir * np.pi / 180)])
                play = True
        if event.type == pygame.QUIT:
            running = False
        
    # Clear screen
    pygame.draw.rect(screen, BROWN, ((WIDTH - SCALE * W[0])//2, (HEIGHT - SCALE * H[0])//2, SCALE * W[0], SCALE * H[0]), border_radius=SCALE * W[0] // 10)
    pygame.draw.rect(screen, GREEN, (X_OFFSET, Y_OFFSET, SCALE * W[1], SCALE * H[1]))
    for p in pockets:
        pygame.draw.circle(screen, BLACK, (p[0], p[1]), SCALE * POCKET_RADIUS)
    
    # Draw arrow
    if not play:
        start = (np.round(cue_pos[0]) + X_OFFSET, np.round(cue_pos[1]) + Y_OFFSET)
        end = np.add(start, SCALE * 10 * np.array([np.cos(dir * np.pi / 180), np.sin(dir * np.pi / 180)]))
        end = tuple(end)
        draw_arrow(screen, pygame.math.Vector2(start), pygame.math.Vector2(end), BLACK, 2, 10, 6)
    
    # Draw balls
    cue = pygame.draw.circle(screen, CUE_COLOR, (np.round(cue_pos[0]) + X_OFFSET, np.round(cue_pos[1]) + Y_OFFSET), SCALE * BALL_RADIUS)
    ball = pygame.draw.circle(screen, BALL_COLOR, (np.round(ball_pos[0]) + X_OFFSET, np.round(ball_pos[1]) + Y_OFFSET), SCALE * BALL_RADIUS)
    
    # Update the screen
    pygame.display.flip()
    
    # Boundary checking
    for pos, vel in zip([cue_pos, ball_pos], [cue_vel, ball_vel]):
        # Check for pocketed balls
        d = np.linalg.norm((pockets - (pos + np.array([X_OFFSET, Y_OFFSET]))) / SCALE, axis=1)
        if (vel > 0).any() and d.min() < POCKET_RADIUS:
            vel *= 0
            if pos is cue_pos:
                CUE_COLOR = RED
            else:
                file.write(f"dir = {dir}, cue_pos = {init_cue_pos}, ball_pos = {init_ball_pos}, pocket = {d.argmin() + 1}\n")
                print(f"\tBall pocketed at pocket {d.argmin() + 1}")
                BALL_COLOR = RED
                
        if pos[0] - SCALE * BALL_RADIUS <= 0:
            pos[0] = SCALE * BALL_RADIUS
            vel[0] *= -1
        if pos[0] + SCALE * BALL_RADIUS >= SCALE * W[1]:
            pos[0] = SCALE * W[1] - SCALE * BALL_RADIUS
            vel[0] *= -1
        if pos[1] - SCALE * BALL_RADIUS <= 0:
            pos[1] = SCALE * BALL_RADIUS
            vel[1] *= -1
        if pos[1] + SCALE * BALL_RADIUS >= SCALE * H[1]:
            pos[1] = SCALE * H[1] - SCALE * BALL_RADIUS
            vel[1] *= -1
            
    # Elastic collision between balls
    if np.linalg.norm(cue_pos - ball_pos) < 2 * SCALE * BALL_RADIUS:
        diff_pos = cue_pos - ball_pos
        change = np.dot(cue_vel - ball_vel, diff_pos) * diff_pos / np.dot(diff_pos, diff_pos)
        cue_vel -= change
        ball_vel += change
    
    # Apply friction to slow down the ball
    cue_vel *= FRIC
    ball_vel *= FRIC
    
    # Update ball positions
    cue_pos += cue_vel
    ball_pos += ball_vel
    
pygame.quit()
file.close()

Initial conditions: dir = 170.0, cue_pos = [115. 365.], ball_pos = [ 93. 369.]


In [3]:
# Read in data randomly
pocketed = []
with open("pocketed.txt", 'r') as file1:
    lines = file1.readlines()
    for line in lines:
        line.strip()
        line = line.split(',')
        line[1] = line[1].replace('. ', '., ')
        line[2] = line[2].replace('. ', '., ')
        pocketed.append([float(line[0]), ast.literal_eval(line[1]), ast.literal_eval(line[2]), int(line[3])])
file1.close()

close_miss = []
with open("close_misses.txt", 'r') as file2:
    lines = file2.readlines()
    for line in lines:
        line.strip()
        line = line.split(',')
        line[1] = line[1].replace('. ', '., ')
        line[2] = line[2].replace('. ', '., ')
        close_miss.append([float(line[0]), ast.literal_eval(line[1]), ast.literal_eval(line[2]), 0])
file2.close()

init_cond = []
# section1 = npr.choice(len(pocketed), 5, replace=False) + 1
pocketed_indices = np.arange(len(pocketed))
miss_indices = np.arange(len(close_miss))
for i in range(20):
    if i < 5 or i >= 10:
        index = rnd.choice(len(pocketed_indices))
        init_cond.append(pocketed[pocketed_indices[index]])
        pocketed_indices = np.delete(pocketed_indices, index)
    else:
        index = rnd.choice(len(miss_indices))
        init_cond.append(close_miss[miss_indices[index]])
        miss_indices = np.delete(miss_indices, index)
    if i == 9:
        random.shuffle(init_cond)

In [4]:
# Load form data
init_cond = []
with open("form.txt", 'r') as file1:
    lines = file1.readlines()
    for line in lines:
        line.strip()
        line = line.split(',')
        line[1] = line[1].replace('. ', '., ')
        line[2] = line[2].replace('. ', '., ')
        init_cond.append([float(line[0]), ast.literal_eval(line[1]), ast.literal_eval(line[2]), int(line[3])])
file1.close()

In [7]:
# Initialize Pygame for form
pygame.init()

# Constants
PADDING = SCALE * (W[0]//15)
WIDTH = SCALE * W[0] + PADDING
HEIGHT = SCALE * H[0] + PADDING
X_OFFSET = (WIDTH - SCALE * W[1])//2
Y_OFFSET = (HEIGHT - SCALE * H[1])//2

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BROWN = (139, 69, 19)
RED = (128, 0, 0)
GREEN = (0, 128, 0)
CUE_COLOR = WHITE
BALL_COLOR = BLACK

# Create background
pygame.display.set_caption("Billiards Simulation")
screen = pygame.display.set_mode((WIDTH, HEIGHT))
screen.fill(WHITE)
pockets = np.zeros((6, 2))
for i in range(6):
    x = (i%2) * SCALE * W[1] + X_OFFSET
    y = (i//2) * SCALE * H[1]//2 + Y_OFFSET
    pockets[i] = np.array([x, y])

# Start simulation
running, init, play = True, True, False
index = 0
while running:
    if init:
        dir, cue_pos, ball_pos = init_cond[index][0], np.array(init_cond[index][1]), np.array(init_cond[index][2])
        init_cue_pos, init_ball_pos = cue_pos.copy(), ball_pos.copy()
        print(f"index = {index+1} dir = {dir}, cue_pos = {cue_pos}, ball_pos = {ball_pos}, pocket = {init_cond[index][3]}")
        index += 1
        cue_vel = np.zeros(2)
        ball_vel = np.zeros(2)
        CUE_COLOR = WHITE
        BALL_COLOR = BLACK
        init = False
        play = False
        
    # keys = pygame.key.get_pressed()
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                init = True
                if index > 20:
                    running = False
            if event.key == pygame.K_RETURN:
                cue_vel = SCALE * MAX_SPEED * np.array([np.cos(dir * np.pi / 180), np.sin(dir * np.pi / 180)])
                play = True
        if event.type == pygame.QUIT:
            running = False
        
    # Clear screen
    pygame.draw.rect(screen, BROWN, ((WIDTH - SCALE * W[0])//2, (HEIGHT - SCALE * H[0])//2, SCALE * W[0], SCALE * H[0]), border_radius=SCALE * W[0] // 10)
    pygame.draw.rect(screen, GREEN, (X_OFFSET, Y_OFFSET, SCALE * W[1], SCALE * H[1]))
    for p in pockets:
        pygame.draw.circle(screen, BLACK, (p[0], p[1]), SCALE * POCKET_RADIUS)
    
    # Draw arrow
    if not play:
        start = (np.round(cue_pos[0]) + X_OFFSET, np.round(cue_pos[1]) + Y_OFFSET)
        end = np.add(start, SCALE * 10 * np.array([np.cos(dir * np.pi / 180), np.sin(dir * np.pi / 180)]))
        end = tuple(end)
        draw_arrow(screen, pygame.math.Vector2(start), pygame.math.Vector2(end), BLACK, 2, 10, 6)
    
    # Draw balls
    cue = pygame.draw.circle(screen, CUE_COLOR, (np.round(cue_pos[0]) + X_OFFSET, np.round(cue_pos[1]) + Y_OFFSET), SCALE * BALL_RADIUS)
    ball = pygame.draw.circle(screen, BALL_COLOR, (np.round(ball_pos[0]) + X_OFFSET, np.round(ball_pos[1]) + Y_OFFSET), SCALE * BALL_RADIUS)
    
    # Update the screen
    pygame.display.flip()
    
    # Boundary checking
    for pos, vel in zip([cue_pos, ball_pos], [cue_vel, ball_vel]):
        # Check for pocketed balls
        d = np.linalg.norm((pockets - (pos + np.array([X_OFFSET, Y_OFFSET]))) / SCALE, axis=1)
        if (vel > 0).any() and d.min() < POCKET_RADIUS:
            vel *= 0
            if pos is cue_pos:
                CUE_COLOR = RED
            else:
                # print(f"\tBall pocketed at pocket {d.argmin() + 1}")
                BALL_COLOR = RED
                
        if pos[0] - SCALE * BALL_RADIUS <= 0:
            pos[0] = SCALE * BALL_RADIUS
            vel[0] *= -1
        if pos[0] + SCALE * BALL_RADIUS >= SCALE * W[1]:
            pos[0] = SCALE * W[1] - SCALE * BALL_RADIUS
            vel[0] *= -1
        if pos[1] - SCALE * BALL_RADIUS <= 0:
            pos[1] = SCALE * BALL_RADIUS
            vel[1] *= -1
        if pos[1] + SCALE * BALL_RADIUS >= SCALE * H[1]:
            pos[1] = SCALE * H[1] - SCALE * BALL_RADIUS
            vel[1] *= -1
            
    # Elastic collision between balls
    if np.linalg.norm(cue_pos - ball_pos) < 2 * SCALE * BALL_RADIUS:
        diff_pos = cue_pos - ball_pos
        change = np.dot(cue_vel - ball_vel, diff_pos) * diff_pos / np.dot(diff_pos, diff_pos)
        cue_vel -= change
        ball_vel += change
    
    # Apply friction to slow down the ball
    cue_vel *= FRIC
    ball_vel *= FRIC
    
    # Update ball positions
    cue_pos += cue_vel 
    ball_pos += ball_vel
    
pygame.quit()

index = 1 dir = -58.0, cue_pos = [145. 326.], ball_pos = [181. 269.], pocket = 0


index = 2 dir = 80.0, cue_pos = [107. 147.], ball_pos = [112. 174.], pocket = 0
index = 3 dir = 31.0, cue_pos = [ 51. 223.], ball_pos = [ 66. 232.], pocket = 3
index = 4 dir = 37.0, cue_pos = [102.  90.], ball_pos = [129. 110.], pocket = 0
index = 5 dir = 118.0, cue_pos = [ 30. 349.], ball_pos = [ 17. 373.], pocket = 4
index = 6 dir = 156.0, cue_pos = [29. 24.], ball_pos = [13. 31.], pocket = 3
index = 7 dir = 47.0, cue_pos = [271. 149.], ball_pos = [293. 173.], pocket = 0
index = 8 dir = 88.0, cue_pos = [201. 460.], ball_pos = [202. 490.], pocket = 0
index = 9 dir = -17.0, cue_pos = [278. 193.], ball_pos = [342. 174.], pocket = 2
index = 10 dir = 24.0, cue_pos = [ 40. 207.], ball_pos = [ 65. 218.], pocket = 4
index = 11 dir = 174.0, cue_pos = [193. 185.], ball_pos = [144. 190.], pocket = 3
index = 12 dir = -70.0, cue_pos = [120. 194.], ball_pos = [130. 167.], pocket = 4
index = 13 dir = -78.0, cue_pos = [ 28. 579.], ball_pos = [ 42. 514.], pocket = 6
index = 14 dir = 23.0, cue_pos = [

: 